In [1]:
# Import findspark and initialize. 
import findspark
findspark.init()

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [10]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

df = spark.read.option("header", "true").option("inferSchema", "true").csv(url)

UnsupportedOperationException: None

In [11]:
# 2. Create a temporary view of the DataFrame.

df.createOrReplaceTempView("home_sales")

NameError: name 'df' is not defined

In [ ]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
avg_price_per_year = spark.sql("""
    SELECT YEAR(Sale_Date) AS Sale_Year, ROUND(AVG(Price), 2) AS Avg_Price
    FROM home_sales
    WHERE Bedrooms = 4
    GROUP BY Sale_Year
    ORDER BY Sale_Year
""")

# Show the result
avg_price_per_year.show()


In [ ]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
avg_price_per_year_built = spark.sql("""
    SELECT Year_Built, ROUND(AVG(Price), 2) AS Avg_Price
    FROM home_sales
    WHERE Bedrooms = 3 AND Bathrooms = 3
    GROUP BY Year_Built
    ORDER BY Year_Built
""")
avg_price_per_year_built.show()


In [ ]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?

avg_price_specific_criteria = spark.sql("""
    SELECT Year_Built, ROUND(AVG(Price), 2) AS Avg_Price
    FROM home_sales
    WHERE Bedrooms = 3
      AND Bathrooms = 3
      AND Floors = 2
      AND Area >= 2000
    GROUP BY Year_Built
    ORDER BY Year_Built
""")
avg_price_specific_criteria.show()

In [ ]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating. 
# Although this is a small dataset, determine the run time for this query.

start_time = time.time()

avg_price_per_view_rating = spark.sql("""
    SELECT View, ROUND(AVG(Price), 2) AS Avg_Price
    FROM home_sales
    GROUP BY View
    HAVING AVG(Price) >= 350000
    ORDER BY View DESC
""")
avg_price_per_view_rating.show()

end_time = time.time()
runtime = end_time - start_time
print(f"--- Query 6 runtime: {runtime} seconds ---")

In [ ]:
# 7. Cache the the temporary table home_sales.
spark.catalog.cacheTable("home_sales")

In [ ]:
# 8. Check if the table is cached.
is_cached = spark.catalog.isCached("home_sales")
print(f"Is home_sales cached? {is_cached}")

In [ ]:
# 9. Using the cached data, run the last query above, that calculates 
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000. 
# Determine the runtime and compare it to the uncached runtime.
start_time_cached = time.time()

avg_price_per_view_rating_cached = spark.sql("""
    SELECT View, ROUND(AVG(Price), 2) AS Avg_Price
    FROM home_sales
    GROUP BY View
    HAVING AVG(Price) >= 350000
    ORDER BY View DESC
""")
avg_price_per_view_rating_cached.show()

end_time_cached = time.time()
runtime_cached = end_time_cached - start_time_cached
print(f"--- Query 9 (cached) runtime: {runtime_cached} seconds ---")


In [ ]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 
df.write.partitionBy("date_built").parquet("path_to_save_formatted_parquet")

In [ ]:
# 11. Read the formatted parquet data.
parquet_df = spark.read.parquet("path_to_save_formatted_parquet")

In [ ]:
# 12. Create a temporary table for the parquet data.
parquet_df.createOrReplaceTempView("parquet_home_sales")

In [ ]:
# 13. Using the parquet DataFrame, run the last query above, that calculates 
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000. 
# Determine the runtime and compare it to the cached runtime.
start_time_parquet = time.time()

avg_price_per_view_rating_parquet = spark.sql("""
    SELECT View, ROUND(AVG(Price), 2) AS Avg_Price
    FROM parquet_home_sales
    GROUP BY View
    HAVING AVG(Price) >= 350000
    ORDER BY View DESC
""")
avg_price_per_view_rating_parquet.show()

end_time_parquet = time.time()
runtime_parquet = end_time_parquet - start_time_parquet
print(f"--- Query 13 (parquet) runtime: {runtime_parquet} seconds ---")

In [ ]:
# 14. Uncache the home_sales temporary table.
spark.catalog.uncacheTable("home_sales")

In [ ]:
# 15. Check if the home_sales is no longer cached
is_cached_after_uncache = spark.catalog.isCached("home_sales")
print(f"Is home_sales cached after uncache? {is_cached_after_uncache}")
